# Exploring Mental Health Discourse on Reddit

### Import Libraries 

In [32]:
import os
import datetime
import time
import re
import requests
import random

import emoji
import pandas as pd
import praw

from nltk.corpus import stopwords

import string 
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist

In [29]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/amyou/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/amyou/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/amyou/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Connecting to Reddit API

*The code was modified based on resources available at https://github.com/rogerfitz/tutorials/blob/master/subreddit_analysis/0_Setup.ipynb and a tutorial accessed via https://www.youtube.com/watch?v=a3zQdlLwEb4.*

In [2]:
# establish connection with reddit API
reddit = praw.Reddit(
    client_id="81TBuwGeFdL5ZfijNpLxNw",
    client_secret="fn2ztcn8ojr3vewO93h46nZ2YY47mw",
    user_agent="Academic_Winner_5303",
)

#### Obtaining Posts from r/mentalhealth

In [3]:
# specify the subreddit
subreddit_mh = 'MentalHealth'

# fetch posts fromsubreddit
subreddit = reddit.subreddit(subreddit_mh)
word_to_check = "mental health"

In [4]:
for submission in reddit.subreddit('mentalhealth').hot(limit=None):
    print(submission.title)

Warning regarding DM's and chatgroups offering "help".
i don't understand why people wants a mental illness when they're completely healthy or is it another kind of illness?
Is there more to life than just being alive?
How do you deal with living with the feeling you destroyed your life, dreams, and hopes?
I feel like nothing 
How frequent do you cry alone? 😔
How do you find hobbies/things to enjoy if you're depressed?
How do I say sorry?
My stepmum is suffering from delusional parasitosis and we don't know what to do
Is it bad to chase a goal that is physical 1000% impossible?
I hate being loved
I think I’m actually stupid 
Family disowned me.
I need someone to talk to rn
I think everyone hates me.
I'm too scared to get help
Does love exist, even within trauma and exhaustion?
Why am I shy to show any interest in myself?
I have been going through a lot i need someone to talk
Very worried returning to work due to depression and anxiety for 5 weeks
Should I change school?
When would you 

In [5]:
subreddit=reddit.subreddit("mentalhealth")
subreddit.over18

False

In [6]:
subreddit.subscribers

465861

In [7]:
subreddit.__dict__

{'display_name': 'mentalhealth',
 '_reddit': <praw.reddit.Reddit at 0x7fb933fb84d0>,
 '_fetched': True,
 '_path': 'r/mentalhealth/',
 'user_flair_background_color': None,
 'submit_text_html': '<!-- SC_OFF --><div class="md"><p>All promotional, survey, research study, or business posts require moderator approval. Please message a moderator to check if your post is suitable. Avoid posting direct links without some context.</p>\n</div><!-- SC_ON -->',
 'restrict_posting': True,
 'user_is_banned': None,
 'free_form_reports': True,
 'wiki_enabled': True,
 'user_is_muted': None,
 'user_can_flair_in_sr': None,
 'header_img': None,
 'title': 'Mental Health',
 'allow_galleries': False,
 'icon_size': None,
 'primary_color': '#005ba1',
 'active_user_count': 41,
 'icon_img': '',
 'display_name_prefixed': 'r/mentalhealth',
 'accounts_active': 41,
 'public_traffic': False,
 'subscribers': 465861,
 'user_flair_richtext': [],
 'name': 't5_2qirg',
 'quarantine': False,
 'hide_ads': False,
 'prediction_

In [8]:
# define function to fetch posts from subreddit ('MentalHealthSupport')
def fetch_posts(subreddit, category='hot', limit=None):
    if category == 'top':
        posts = subreddit.top(time_filter='year', limit=limit)
    elif category == 'rising':
        posts = subreddit.rising(limit=limit)
    elif category == 'new':
        posts = subreddit.new(limit=limit)
    else:
        posts = subreddit.hot(limit=limit)
    
    rows = []
    for submission in posts:
        row = {
            'title': submission.title,
            'score': submission.score,
            'upvote_ratio': submission.upvote_ratio,
            'created_utc': submission.created_utc,
            'selftext': submission.selftext,
            'subreddit': submission.subreddit,
            'author': str(submission.author),
            'media_only': submission.media_only,
            'permalink': submission.permalink,
            'num_comments': submission.num_comments
        }
        rows.append(row)
    
    return pd.DataFrame(rows)

In [9]:
mental_health_top = fetch_posts(subreddit, category='top')
mental_health_hot = fetch_posts(subreddit, category='hot')
mental_health_rising = fetch_posts(subreddit, category='rising')
mental_health_new = fetch_posts(subreddit, category='new')

In [10]:
# concatenate dataframes
mental_health_df = pd.concat([mental_health_top,
                               mental_health_hot, 
                               mental_health_rising, 
                               mental_health_new],
                              ignore_index=True)

# drop duplicates
mental_health_df = mental_health_df.drop_duplicates()

In [11]:
mental_health_df['upvotes']=mental_health_df['score']/mental_health_df['upvote_ratio']
mental_health_df['downvotes']=mental_health_df['upvotes']-mental_health_df['score']
mental_health_df['upvotes/subscribers']=mental_health_df['upvotes']/subreddit.subscribers

mental_health_df.sample(5)

,title,score,upvote_ratio,created_utc,selftext,subreddit,author,media_only,permalink,num_comments,upvotes,downvotes,upvotes/subscribers
623,why am i so jealous and selfish and how do i stop,53,0.98,1.688141e+09,im not selfish in like terms of money or mater...,mentalhealth,hash4ash,False,/r/mentalhealth/comments/14n4cje/why_am_i_so_j...,33,54.081633,1.081633,0.000116
1194,Anxiety disorder,1,1.00,1.717228e+09,I have pretty severe social anxiety disorder b...,mentalhealth,cobaltcodebrew,False,/r/mentalhealth/comments/1d5hulh/anxiety_disor...,0,1.000000,0.000000,0.000002
292,All we do is cope,113,0.98,1.683055e+09,"Whatever are you doing right now, working, stu...",mentalhealth,IfallInLove2easily,False,/r/mentalhealth/comments/135xlnn/all_we_do_is_...,61,115.306122,2.306122,0.000248
1277,Has anyone experienced this?,1,1.00,1.717198e+09,My elderly relative that I care for is diagnos...,mentalhealth,Emergency_Camel7605,False,/r/mentalhealth/comments/1d59b4c/has_anyone_ex...,0,1.000000,0.000000,0.000002
1827,What is it and is it serious?,1,1.00,1.716965e+09,Recentely some of my weird behaviours and thou...,mentalhealth,_Just_asking_stuff_,False,/r/mentalhealth/comments/1d35tcm/what_is_it_an...,3,1.000000,0.000000,0.000002


In [12]:
mental_health_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2055 entries, 0 to 2927
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   title                2055 non-null   object 
 1   score                2055 non-null   int64  
 2   upvote_ratio         2055 non-null   float64
 3   created_utc          2055 non-null   float64
 4   selftext             2055 non-null   object 
 5   subreddit            2055 non-null   object 
 6   author               2055 non-null   object 
 7   media_only           2055 non-null   bool   
 8   permalink            2055 non-null   object 
 9   num_comments         2055 non-null   int64  
 10  upvotes              2055 non-null   float64
 11  downvotes            2055 non-null   float64
 12  upvotes/subscribers  2055 non-null   float64
dtypes: bool(1), float64(5), int64(2), object(5)
memory usage: 210.7+ KB


In [13]:
# create csv 
mental_health_df.to_csv('mental_health_posts.csv', index=False)

#### Obtaining Posts  from r/MentalHealthSupport 

In [14]:
# specify the subreddit
subreddit_mhs = 'MentalHealthSupport'

# fetch posts fromsubreddit
subreddit = reddit.subreddit(subreddit_mhs)
word_to_check = "mental health"

In [15]:
for submission in reddit.subreddit('mentalhealthsupport').hot(limit=None):
    print(submission.title)

i give up
Think I'm being manipulated by a narcissist 
Spider-Verse Mental Health Awreness short needed some work.
Is this normal?
What’s possible to create with KI?
Self Sabotaging Relationships 
Dealing with p*dos
How do you get past this?
what's a thing you regret the most as a kid?
is it offensive if i ask my partner to go to therapy?
My mental health is worse because of a pedophile.
Motivation
What is wrong with me?
I'm Empty
post exam anxiety/depression
Need help, I'v been unable to feel Strong emotions or Any sense of urgency For the past four months. What do?
Tell me it gets better
Teen mental health
Something very wrong but I don’t know what it’s is
Very scared of brain eating ameoba after tap water from work went into my nose.
Breaking Free from Conditioning: Seeing Beyond the Frame
God I hate my social anxiety disorder!
mental health help
The push and pull
Laziness -> Overthinking -> Inferiority -> Depression 
Going back to school as a "mature student" and feeling weird abou

In [16]:
subreddit=reddit.subreddit('mentalhealthsupport')
subreddit.over18

False

In [17]:
subreddit.subscribers

45597

In [18]:
subreddit.__dict__

{'display_name': 'MentalHealthSupport',
 '_reddit': <praw.reddit.Reddit at 0x7fb933fb84d0>,
 '_fetched': True,
 '_path': 'r/MentalHealthSupport/',
 'user_flair_background_color': None,
 'submit_text_html': '<!-- SC_OFF --><div class="md"><p>Please provide context to links since they require moderator approval. Promotional posts are not permitted.  Message the moderators if you have any query. Make sure your post follows the rules.</p>\n\n<p>This subreddit is optimized for the new Reddit redesign and official app. Please use the newer design to enjoy the subreddit to its fullest, thank you!</p>\n</div><!-- SC_ON -->',
 'restrict_posting': True,
 'user_is_banned': None,
 'free_form_reports': True,
 'wiki_enabled': True,
 'user_is_muted': None,
 'user_can_flair_in_sr': None,
 'header_img': None,
 'title': 'Mental Health Support & Azal',
 'allow_galleries': False,
 'icon_size': None,
 'primary_color': '#2a4fd5',
 'active_user_count': 12,
 'icon_img': '',
 'display_name_prefixed': 'r/Mental

In [19]:
# define function to fetch posts from subreddit ('MentalHealthSupport')
def fetch_posts(subreddit, category='hot', limit=None):
    if category == 'top':
        posts = subreddit.top(time_filter='year', limit=limit)
    elif category == 'rising':
        posts = subreddit.rising(limit=limit)
    elif category == 'new':
        posts = subreddit.new(limit=limit)
    else:
        posts = subreddit.hot(limit=limit)
    
    rows = []
    for submission in posts:
        row = {
            'title': submission.title,
            'score': submission.score,
            'upvote_ratio': submission.upvote_ratio,
            'created_utc': submission.created_utc,
            'selftext': submission.selftext,
            'subreddit': submission.subreddit,
            'author': str(submission.author),
            'media_only': submission.media_only,
            'permalink': submission.permalink,
            'num_comments': submission.num_comments
        }
        rows.append(row)
    
    return pd.DataFrame(rows)

In [20]:
mental_health_support_top = fetch_posts(subreddit, category='top')
mental_health_support_hot = fetch_posts(subreddit, category='hot')
mental_health_support_rising = fetch_posts(subreddit, category='rising')
mental_health_support_new = fetch_posts(subreddit, category='new')

In [21]:
# concatenate dataframes
mental_health_support_df = pd.concat([mental_health_support_top, 
                                      mental_health_support_hot, 
                                      mental_health_support_rising, 
                                      mental_health_support_new], 
                                     ignore_index=True)

# drop duplicates
mental_health_support_df = mental_health_support_df.drop_duplicates()

In [22]:
mental_health_support_df.head()

,title,score,upvote_ratio,created_utc,selftext,subreddit,author,media_only,permalink,num_comments
0,I feel hopeless.,157,0.99,1.709293e+09,"Hi, I’m 24F. I stay home 24/7 , everyday. I do...",MentalHealthSupport,Odd-View-667,False,/r/MentalHealthSupport/comments/1b3s6ad/i_feel...,154
1,What changes your negative thoughts/ has made ...,47,1.00,1.709444e+09,I am learning that affirmations and the like c...,MentalHealthSupport,Bananasloog1998,False,/r/MentalHealthSupport/comments/1b59lzz/what_c...,21
2,what is the point of living?,46,0.94,1.709354e+09,\ncontent warning for suicide.\n\n\n\nI dont u...,MentalHealthSupport,penjamincranklin,False,/r/MentalHealthSupport/comments/1b4fkko/what_i...,24
3,I need a hug,23,0.90,1.709557e+09,I wanna end me.,MentalHealthSupport,Positive-Parthlow,False,/r/MentalHealthSupport/comments/1b6a8vz/i_need...,26
4,What are you like after a hug,22,1.00,1.709738e+09,If I hug someone I just try my best not to cry...,MentalHealthSupport,itsRileyigitbanned,False,/r/MentalHealthSupport/comments/1b8299d/what_a...,17


In [23]:
# calculate upvotes and downvotes
mental_health_support_df['upvotes']=mental_health_support_df['score']/mental_health_support_df['upvote_ratio']
mental_health_support_df['downvotes']=mental_health_support_df['upvotes']-mental_health_support_df['score']
mental_health_support_df['upvotes/subscribers']=mental_health_support_df['upvotes']/subreddit.subscribers

mental_health_support_df.sample(5)

,title,score,upvote_ratio,created_utc,selftext,subreddit,author,media_only,permalink,num_comments,upvotes,downvotes,upvotes/subscribers
129,"Please help me, my father is not in good shape!",5,1.0,1.711272e+09,"Hey, long story short, about last week on Thru...",MentalHealthSupport,ANBUGaming19,False,/r/MentalHealthSupport/comments/1bmgmeu/please...,1,5.0,0.0,0.000110
1151,It's been a hard few months,1,1.0,1.716527e+09,Hey all newbie here but seriously struggling ...,MentalHealthSupport,According_Cut814,False,/r/MentalHealthSupport/comments/1czczhx/its_be...,0,1.0,0.0,0.000022
130,I feel like I'm too numb and don't mentally re...,5,1.0,1.711252e+09,TW: SA? So idk why im posting this but I'm jus...,MentalHealthSupport,Afraid-Fishing-3175,False,/r/MentalHealthSupport/comments/1bmbgao/i_feel...,2,5.0,0.0,0.000110
1833,Stuck in a hole,1,1.0,1.714356e+09,I want to tell someone a story about myself th...,MentalHealthSupport,Freecs_27,False,/r/MentalHealthSupport/comments/1cfnmjd/stuck_...,0,1.0,0.0,0.000022
165,"I feel so alone, everything is so loud and I j...",6,1.0,1.709605e+09,"I'm 18, the only reason I'm still around is it...",MentalHealthSupport,Someseed,False,/r/MentalHealthSupport/comments/1b6ts3i/i_feel...,4,6.0,0.0,0.000132


In [24]:
# create csv 
mental_health_support_df.to_csv('mental_health_support_posts.csv', index=False)

In [25]:
mental_health_support_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1916 entries, 0 to 2835
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   title                1916 non-null   object 
 1   score                1916 non-null   int64  
 2   upvote_ratio         1916 non-null   float64
 3   created_utc          1916 non-null   float64
 4   selftext             1916 non-null   object 
 5   subreddit            1916 non-null   object 
 6   author               1916 non-null   object 
 7   media_only           1916 non-null   bool   
 8   permalink            1916 non-null   object 
 9   num_comments         1916 non-null   int64  
 10  upvotes              1916 non-null   float64
 11  downvotes            1916 non-null   float64
 12  upvotes/subscribers  1916 non-null   float64
dtypes: bool(1), float64(5), int64(2), object(5)
memory usage: 196.5+ KB


#### Combine Dataframes (r/MentalHealth and  r/MentalHealthSupport)

In [26]:
# combine both df into one
df = pd.concat([mental_health_support_df, 
                mental_health_df],
               ignore_index=True)

In [27]:
# create csv 
df.to_csv('reddit_posts.csv', index=False)

In [28]:
df.info()

df.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3971 entries, 0 to 3970
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   title                3971 non-null   object 
 1   score                3971 non-null   int64  
 2   upvote_ratio         3971 non-null   float64
 3   created_utc          3971 non-null   float64
 4   selftext             3971 non-null   object 
 5   subreddit            3971 non-null   object 
 6   author               3971 non-null   object 
 7   media_only           3971 non-null   bool   
 8   permalink            3971 non-null   object 
 9   num_comments         3971 non-null   int64  
 10  upvotes              3971 non-null   float64
 11  downvotes            3971 non-null   float64
 12  upvotes/subscribers  3971 non-null   float64
dtypes: bool(1), float64(5), int64(2), object(5)
memory usage: 376.3+ KB


,title,score,upvote_ratio,created_utc,selftext,subreddit,author,media_only,permalink,num_comments,upvotes,downvotes,upvotes/subscribers
492,I’ve been suffering with mental issues for 15 ...,2,1.00,1.711972e+09,I’m 31 now and I’ve lost so much due to then\n...,MentalHealthSupport,None,False,/r/MentalHealthSupport/comments/1bt1eyl/ive_be...,2,2.000000,0.000000,0.000044
850,What is it exactly about customer service that...,1,0.54,1.710964e+09,Most of the customers I deal with are not rude...,MentalHealthSupport,The-Loop,False,/r/MentalHealthSupport/comments/1bjmkji/what_i...,2,1.851852,0.851852,0.000041
1436,"I need help, as a Canadian where is the best p...",1,1.00,1.715663e+09,I think I have some serious issues that needs ...,MentalHealthSupport,Shusmita_Dipti,False,/r/MentalHealthSupport/comments/1crk9o9/i_need...,0,1.000000,0.000000,0.000022
1883,How do you find purpose In our bleak reality?,13,1.00,1.714672e+09,The many minor achievements and no major I fee...,MentalHealthSupport,That1NerdNoel,False,/r/MentalHealthSupport/comments/1ciltyp/how_do...,7,13.000000,0.000000,0.000285
3582,So tired of my life want it to be over.,2,1.00,1.717025e+09,And I just can’t get the words out and it hurts,mentalhealth,Sensitive-Push-5926,False,/r/mentalhealth/comments/1d3podp/so_tired_of_m...,2,2.000000,0.000000,0.000004


Tokenization/normalization

In [33]:
# Tokenization and Normalization
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


In [34]:
def preprocess_text(text):
    # Tokenize
    tokens = word_tokenize(text.lower())
    # Remove stop words and punctuation
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    # Lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

In [35]:
# Apply preprocessing to the 'selftext' column
df['tokens'] = df['selftext'].apply(lambda x: preprocess_text(x) if isinstance(x, str) else [])

# Descriptive stats
df['token_count'] = df['tokens'].apply(len)
numerical_stats = df.describe()
token_count_stats = df['token_count'].describe()

# Most common words
all_tokens = [token for tokens in df['tokens'] for token in tokens]
fdist = FreqDist(all_tokens)
most_common_words = fdist.most_common(10)

# print stats
print("Numerical Statistics:\n", numerical_stats)
print("\nToken Count Statistics:\n", token_count_stats)
print("\nMost Common Words:\n", most_common_words)

Numerical Statistics:
              score  upvote_ratio   created_utc  num_comments      upvotes  \
count  3971.000000   3971.000000  3.971000e+03   3971.000000  3971.000000   
mean     30.184085      0.961770  1.710683e+09     17.330395    31.679307   
std      72.574160      0.088231  8.971526e+06     41.159640    75.609138   
min       0.000000      0.250000  1.680765e+09      0.000000     0.000000   
25%       1.000000      0.970000  1.709689e+09      0.000000     1.000000   
50%       2.000000      1.000000  1.714821e+09      2.000000     2.000000   
75%      23.000000      1.000000  1.716998e+09     13.000000    23.355670   
max     925.000000      1.000000  1.717316e+09    520.000000   943.877551   

         downvotes  upvotes/subscribers  token_count  
count  3971.000000          3971.000000  3971.000000  
mean      1.495223             0.000096    95.616973  
std       4.815449             0.000171   104.177741  
min       0.000000             0.000000     0.000000  
25%     